In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
import mlflow
from tqdm import tqdm
from itertools import product

In [2]:
# MovieLens 데이터 로드
path = "../data/raw/ml-latest-small/"
ratings = pd.read_csv(path + "ratings.csv")
movies = pd.read_csv(path + "movies.csv")

ratings = ratings[ratings['rating'] >=   3.5]

user_ids = ratings['userId'].unique().tolist()
movie_ids = ratings['movieId'].unique().tolist()
user_to_idx = {user_id: idx for idx, user_id in enumerate(user_ids)}
movie_to_idx = {movie_id: idx for idx, movie_id in enumerate(movie_ids)}

ratings['user_idx'] = ratings['userId'].map(user_to_idx)
ratings['movie_idx'] = ratings['movieId'].map(movie_to_idx)

num_users = len(user_ids)
num_items = len(movie_ids)

# 학습/테스트 데이터 분리
train_ratings, test_ratings = train_test_split(ratings, test_size=0.2, random_state=42)

print(f"Number of users: {num_users}, Number of movies: {num_items}")
print(f"Train size: {len(train_ratings)}, Test size: {len(test_ratings)}")

Number of users: 609, Number of movies: 7363
Train size: 49372, Test size: 12344


In [3]:
# NCF용 데이터셋 클래스
class MovieLensNCFDataset(Dataset):
    def __init__(self, ratings_df, num_items, num_negatives=3):
        self.users, self.items, self.labels = self._get_dataset_optimized(ratings_df, num_items, num_negatives)

    # 기본 구현 (교육용))    
    def _get_dataset(self, ratings_df, num_items, num_negatives):
        users, items, labels = [], [], []
        user_item_set = set(zip(ratings_df['user_idx'], ratings_df['movie_idx']))

        for (u, i) in user_item_set:
            users.append(u)
            items.append(i)
            labels.append(1.0)

            for _ in range(num_negatives):
                neg_item = np.random.randint(num_items)
                while (u, neg_item) in user_item_set:
                    neg_item = np.random.randint(num_items)
                users.append(u)
                items.append(neg_item)
                labels.append(.0)

        return torch.tensor(users, dtype=torch.long), torch.tensor(items, dtype=torch.long), torch.tensor(labels, dtype=torch.float32)

    # 벡터화된 구현 (하지만 negative 샘플링이 완전하지 않음, 실제로 positive 샘플이 포함될 수 있음)
    def _get_dataset_vectorized(self, ratings_df, num_items, num_negatives):
        user_pos = ratings_df['user_idx'].values
        item_pos = ratings_df['movie_idx'].values
        label_pos= np.ones(len(user_pos), dtype=np.float32)

        user_neg = np.repeat(user_pos, num_negatives)
        item_neg = np.random.randint(0, num_items, size=len(user_neg))
        label_neg = np.zeros(len(user_neg), dtype=np.float32)

        users = np.concatenate([user_pos, user_neg])
        items = np.concatenate([item_pos, item_neg])
        labels = np.concatenate([label_pos, label_neg])

        return torch.tensor(users, dtype=torch.long), torch.tensor(items, dtype=torch.long), torch.tensor(labels, dtype=torch.float32)
    
    # 최적화된 구현 (negative 샘플링이 완전하지는 않을 확률이 극히 적음, 사실상 완전하다고 봐도 됨)
    def _get_dataset_optimized(self, ratings_df, num_items, num_negatives):
        user_pos = ratings_df['user_idx'].values
        item_pos = ratings_df['movie_idx'].values
        label_pos = np.ones(len(user_pos), dtype=np.float32)

        pos_hashed = user_pos * num_items + item_pos

        user_neg = np.repeat(user_pos, num_negatives)
        item_neg = np.random.randint(0, num_items, size=len(user_neg))
        label_neg = np.zeros(len(user_neg), dtype=np.float32)
        
        for _ in range(5):
            neg_hashed = user_neg * num_items + item_neg

            mask = np.isin(neg_hashed, pos_hashed)

            if np.sum(mask) == 0:
                break
            
            item_neg[mask] = np.random.randint(0, num_items, size=mask.sum())
        
        users = np.concatenate([user_pos, user_neg])
        items = np.concatenate([item_pos, item_neg])
        labels = np.concatenate([label_pos, label_neg])

        return torch.tensor(users, dtype=torch.long), torch.tensor(items, dtype=torch.long), torch.tensor(labels, dtype=torch.float32)
        
    
    def __len__(self):
        return len(self.users)
    
    def __getitem__(self, idx):
        return self.users[idx], self.items[idx], self.labels[idx]

# 데이터셋 생성
train_dataset = MovieLensNCFDataset(ratings_df=train_ratings, num_items=num_items, num_negatives=6)
test_dataset = MovieLensNCFDataset(ratings_df=test_ratings, num_items=num_items, num_negatives=6)
print(f"훈련 데이터셋 크기: {len(train_dataset)}")
print(f"테스트 데이터셋 크기: {len(test_dataset)}")

훈련 데이터셋 크기: 345604
테스트 데이터셋 크기: 86408


In [4]:
# 배치 사이즈
BATCH_SIZE = 256

# 학습 데이터로더 생성
train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
)

# 테스트 데이터로더 생성
test_loader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
)

data_iter = iter(train_loader)
user_batch, item_batch, label_batch = next(data_iter)


print("User Batch Shape:", user_batch.shape)
print("Item Batch Shape:", item_batch.shape)
print("Label Batch Shape:", label_batch.shape)

User Batch Shape: torch.Size([256])
Item Batch Shape: torch.Size([256])
Label Batch Shape: torch.Size([256])


In [5]:
# NCF Two Tower Model
class NCFModel(nn.Module):
    def __init__(self, num_users, num_items, latent_dim=32):
        super().__init__()

        # 임베딩 레이어 (Lookup Table)
        self.user_embedding = nn.Embedding(num_users, latent_dim)
        self.item_embedding = nn.Embedding(num_items, latent_dim)

        nn.init.normal_(self.user_embedding.weight, std=0.01)
        nn.init.normal_(self.item_embedding.weight, std=0.01)

    def forward(self, user_indices, item_indices):
        user_vec = self.user_embedding(user_indices)
        item_vec = self.item_embedding(item_indices)

        # 배치 차원에서 두 벡터간의 내적 계산
        dot_product = (user_vec * item_vec).sum(dim=1)

        return dot_product

In [6]:
# cpu, cuda, mps 중 사용 가능한 디바이스 선택
device = "cpu"
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")

# 하이퍼파라미터 설정
param_grid = {
    "epochs": [5, 10],
    "lr": [0.001, 0.0001],
    "batch_size": [512, 1024],
    "latent_dim": [32, 64, 128],
    "num_negatives": [4]
}

keys = param_grid.keys()
# 모든 하이퍼파라미터 조합 생성
combinations = list(product(*param_grid.values()))

mlflow.set_experiment("NCF_MovieLens_Recsys")

for i, values in enumerate(combinations):
    params = dict(zip(keys, values))
    run_name = f"NCF_lr{params['lr']}_bs{params['batch_size']}_dim{params['latent_dim']}"
    print(f"\n[{i+1}/{len(combinations)}] {run_name} 시작!!!")
    
    train_loader = DataLoader(train_dataset, batch_size=params["batch_size"], shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=params["batch_size"], shuffle=False)

    model = NCFModel(num_users=num_users, num_items=num_items, latent_dim=params["latent_dim"])
    model = model.to(device)

    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=params["lr"])


    with mlflow.start_run(run_name=run_name):
        mlflow.log_params(params)
        best_loss = float('inf')

        for epoch in range(params["epochs"]):
            # 학습
            model.train()
            epoch_train_loss = 0.0
            pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{params['epochs']} [Train]")
            for user_batch, item_batch, label_batch in pbar:
                user_batch, item_batch, label_batch = user_batch.to(device), item_batch.to(device), label_batch.to(device)
                
                optimizer.zero_grad()
                outputs = model(user_batch, item_batch)
                loss = criterion(outputs, label_batch)
                loss.backward()
                optimizer.step()

                epoch_train_loss += loss.item()
                pbar.set_postfix({"loss": loss.item()})

            avg_train_loss = epoch_train_loss / len(train_loader)
            mlflow.log_metric("train_loss", avg_train_loss, step=epoch)
            
            # 테스트
            model.eval()
            epoch_test_loss = 0.0
            with torch.no_grad():
                pbar_test = tqdm(test_loader, desc=f"Epoch {epoch+1}/{params['epochs']} [Test]", leave=False)
                for user_batch, item_batch, label_batch in pbar_test:
                    user_batch, item_batch, label_batch = user_batch.to(device), item_batch.to(device), label_batch.to(device)
                    outputs = model(user_batch, item_batch)
                    loss = criterion(outputs, label_batch)
                    epoch_test_loss += loss.item()
            
            avg_test_loss = epoch_test_loss / len(test_loader)
            mlflow.log_metric("test_loss", avg_test_loss, step=epoch)
            
            print(f"Epoch {epoch+1}: Train Loss: {avg_train_loss:.4f}, Test Loss: {avg_test_loss:.4f}")

            if avg_test_loss < best_loss:
                best_loss = avg_test_loss

        mlflow.log_metric("best_test_loss", best_loss)
        mlflow.pytorch.log_model(model, "model")
        print(f"Best Test loss: {best_loss:.4f}")
        
print("모든 실험 완료")

2026/01/11 20:54:43 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2026/01/11 20:54:43 INFO mlflow.store.db.utils: Updating database tables
2026/01/11 20:54:43 INFO alembic.runtime.migration: Context impl SQLiteImpl.
2026/01/11 20:54:43 INFO alembic.runtime.migration: Will assume non-transactional DDL.
2026/01/11 20:54:43 INFO alembic.runtime.migration: Running upgrade  -> 451aebb31d03, add metric step
2026/01/11 20:54:43 INFO alembic.runtime.migration: Running upgrade 451aebb31d03 -> 90e64c465722, migrate user column to tags
2026/01/11 20:54:43 INFO alembic.runtime.migration: Running upgrade 90e64c465722 -> 181f10493468, allow nulls for metric values
2026/01/11 20:54:43 INFO alembic.runtime.migration: Running upgrade 181f10493468 -> df50e92ffc5e, Add Experiment Tags Table
2026/01/11 20:54:43 INFO alembic.runtime.migration: Running upgrade df50e92ffc5e -> 7ac759974ad8, Update run tags with larger limit
2026/01/11 20:54:43 INFO alembic.runtime.migration: Running 


[1/24] NCF_lr0.001_bs512_dim32 시작!!!


Epoch 1/5 [Train]: 100%|██████████| 676/676 [00:03<00:00, 177.72it/s, loss=0.412]


Epoch 1: Train Loss: 0.5251, Test Loss: 0.3403


Epoch 2/5 [Train]: 100%|██████████| 676/676 [00:03<00:00, 183.81it/s, loss=0.179]


...

Epoch 9: Train Loss: 0.3199, Test Loss: 0.3345


Epoch 10/10 [Train]: 100%|██████████| 338/338 [00:03<00:00, 95.66it/s, loss=0.275] 
2026/01/11 21:08:52 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


Epoch 10: Train Loss: 0.3069, Test Loss: 0.3255


2026/01/11 21:08:55 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


Best Test loss: 0.3255
모든 실험 완료
